# Логи запуска в notebook будут недоступны т. к. с 1 декабря 2023 web-интерфейс tensorboard dev приостаналивает работу


# Часть первая: крестики-нолики при помощи Q-обучения

Почти везде графики обучения и результаты будут приведены в Tensorboard. Их можно открыть по ссылке и посмотреть в интерактивном режиме.

При разгрузке были какие-то технические ошибки, поэтому некоторые графики обрываются раньше других. 

## Противники

Для окружения размера 3x3 был построено минимакс дерево с альфа-бета отсечением. Для окружения большего размера была попытка построить MCTS дерево и использовать его в качестве противника при обучении всех алгоритмов. 

Для окружения размера 4x4 результаты по игре оказались не сильно лучше чем при случайном противнике, но вычислительно MCTS дерево оказалось намного тяжелее, поэтому было принято решение использовать случайную стратегию, чтобы быстрее считать и меньше места занимать в оперативной памяти. 

Для окружения 4x4 при полном расширении требовалось слишком много оперативной памяти. Пришлось урезать глубину дерева и не делать полное расширение на всевозможные состояния. Для полного построения MCTS дерева был ещё проверен вариант с построением графа вместо дерева, чтобы переиспользовать одинаковые позиции на поле, но библиотека для работы с графами оказалась намного медленнее, чем с деревьями.

Минимакс дерево было немного рандомизированно, чтобы периодически совершать случайные действия и не всегда следовать оптимальной стратегии.

Для проверки правильности построения и работы минимакс дерева проверялся фак того что если оба игрока следуют минимакс стратегии, то всегда получается ничья.

In [81]:
import pathlib
import pandas as pd

from IPython.display import HTML

In [82]:
%load_ext autoreload
%autoreload 2

from notebook_utils import show_tables, game_stat_to_table

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Результаты игры рандомизированного минимакс дерева против случайного противника.

In [83]:
stat_dir = pathlib.Path("./stat/cross_minmax_vs_random")

### Минимакс за крестиков против случайно стратегии

In [84]:
show_tables(stat_dir)

Вероятность совершить случайное действие,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0.9,0.6245,0.268,0.1075
0.7,0.6775,0.205,0.1175
0.1,0.923,0.0365,0.0405
0.2,0.8505,0.0855,0.064
0,0.979,0,0.021
0.5,0.753,0.164,0.083


### Минимакс за ноликов против случайно стратегии

In [85]:
stat_dir = pathlib.Path("./stat/cross_random_vs_minmax")
show_tables(stat_dir)

Вероятность совершить случайное действие,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0.9,0.5585,0.3235,0.118
0.7,0.506,0.3555,0.1385
0.1,0.175,0.6695,0.1555
0.2,0.2705,0.5965,0.133
0,0,0.7965,0.2035
0.5,0.4405,0.439,0.1205


В итоге была выбрана вероятность равная 0,09, чтобы дать возможность иногда побеждать противнику. 

Далее структура отчёты будет одинакова:
* ссылка на результаты обучения при различных гиперпараметрах (представлены результаты игр);
* ссылка на графики с общей динамикой при различных параметрах;
* ссылка на динамику обучения с наилучшими параметрами.

Описание параметров:
* `valid_cross_win` - доля побед за крестиков после обучения;
* `valid_circle_win` - доля побед за ноликов после обучения;
* `valid_draw` - доля ничьих после обучения.

Значение на графиках с префиксом `Train` это значения аналогичных метрик только во время обучения для отслеживания общей динамики.


* `alpha` - в случае нейронных сетей learning_rate метода оптимизации;
* `train_mean_reward`- это усреднённая награда во время обучения.

**В Q-обучении везде использовалась epsilon-greedy стратегия с epsilon = 0,02. Это могло повлиять иногда на результаты игр** 

## Табличное Q-обучение

### Окружение 3x3

#### Крестики используют Q-обучение - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#scalars&_smoothingWeight=0.999&regexInput=0_)

#### Крестики рандомизированный минимакс - нолики Q-обучение  

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#scalars&_smoothingWeight=0.999&regexInput=2_)

### Окружение 4x4

#### Крестики используют Q-обучение - нолики случайная стратегия

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#scalars&_smoothingWeight=0.999&regexInput=1_)

#### Крестики используют случайную стратегию - нолики Q-обучение

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#scalars&_smoothingWeight=0.999&regexInput=6_)

Тут появились проблемы, если смотреть на графики крупным планом, то некоторая сходимость наблюдается, но очень медленная. Возможно, нужно было намного больше эпизодов, чтобы покрыть большое число состояний и сделать какую-то оценку для дискретных значений Q-функции. 

За ноликов оказалось совсем сложно обучать.

# Часть вторая DQN и Dueling DQN 

Для DQN и Dueling DQN добавляется дополнительная метрика это `L1_loss`. Значение функции потерь, которая использовалась для оптимизации.

## DQN

### Окружение 3x3

#### Крестики используют DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg4NjE4Mi41NjQ3NDQ3Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODg2MzcxLjM0MzMzODMiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg4NjIwNi4yOTMzNDgzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg4NjM3OC4wMTg4OTAxIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzg4NjA2My40OTQ4NjciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4ODYwMzQuNjYxNzk5NCI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc4ODYxNzkuODExMTY1OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc4ODYwMjQuOTQwMzE5NSI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg4NjM3MS4wMTc5MDIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#scalars&regexInput=2_&_smoothingWeight=0.999)

#### Нолики используют DQN - крестики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDQwNy42MDcyNDIzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0NDMzLjcxNjk0MDYiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg5NDQwNC4xMTUzOTQiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM3ODk0MzY5LjcxMTQ3ODUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0MjYyLjA4Mzg1MTgiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4OTQyMTcuMDMyMTkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDMwNy42NTQ1NTQ0Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzNzg5NDQ4Mi43NTgzMTg3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3ODk0MTc3LjQ3NDA5MzIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDQwNy42MDcyNDIzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0NDMzLjcxNjk0MDYiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg5NDQwNC4xMTUzOTQiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM3ODk0MzY5LjcxMTQ3ODUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0MjYyLjA4Mzg1MTgiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4OTQyMTcuMDMyMTkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDMwNy42NTQ1NTQ0Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzNzg5NDQ4Mi43NTgzMTg3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3ODk0MTc3LjQ3NDA5MzIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&regexInput=%5E2_)

Для ноликов с DQN обучение оказалось намного сложнее. Результат постепенно улучшается, но похоже на то что нужно намного больше эпизодов сделать. 

### Окружение 4x4

#### Крестики используют DQN - нолики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzkxNTY4My4zNDQ3ODM1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzA2Ljg2MDk0NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFwxNjM3OTE1Nzk2LjAyOTUyNDgiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NjQxLjk5MjAwNSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXDE2Mzc5MTU4MDYuNTIwODYzOCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzkxNTY1OS4wMTQxMzYiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NzY2LjM1OTAwNTUiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzIwLjA0NjQ5MjYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5MTU3MzcuMjUxMTgyMyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzkxNTY4My4zNDQ3ODM1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzA2Ljg2MDk0NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFwxNjM3OTE1Nzk2LjAyOTUyNDgiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NjQxLjk5MjAwNSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXDE2Mzc5MTU4MDYuNTIwODYzOCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzkxNTY1OS4wMTQxMzYiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NzY2LjM1OTAwNTUiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzIwLjA0NjQ5MjYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5MTU3MzcuMjUxMTgyMyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999&regexInput=%5E5_)

#### Нолики используют DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzkxOTE5OC42OTc1NCI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIzOS45NzQyMDY3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMDQuODgwMDUyOCI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkyMDguMTk0ODQzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIwMS4wMzI1NzQiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMjguOTcxODkzMyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkwNjMuNjkwMDA2NyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5MTkwMzQuNDYzMjExIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTE5MjQyLjYwNjA2NSI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzkxOTE5OC42OTc1NCI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIzOS45NzQyMDY3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMDQuODgwMDUyOCI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkyMDguMTk0ODQzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIwMS4wMzI1NzQiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMjguOTcxODkzMyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkwNjMuNjkwMDA2NyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5MTkwMzQuNDYzMjExIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTE5MjQyLjYwNjA2NSI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999&regexInput=%5E0_)

## Dueling DQN 

### Окружение 3x3

#### Крестики используют Dueling DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjU1MC44NTA2MTAzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNTg5LjQyMTE3MDciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjY3MS43NzUxMTg4Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjgwOC4xMTQxMzIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyMzU5Ljg5MDAzMyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjQxNC41MTczNTIiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTYyNzc4LjcyNjMxNzIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNzQwLjUyMjQ0MDQiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5NjI0MTguMzIxNjM4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjU1MC44NTA2MTAzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNTg5LjQyMTE3MDciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjY3MS43NzUxMTg4Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjgwOC4xMTQxMzIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyMzU5Ljg5MDAzMyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjQxNC41MTczNTIiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTYyNzc4LjcyNjMxNzIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNzQwLjUyMjQ0MDQiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5NjI0MTguMzIxNjM4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999&regexInput=%5E1_)

#### Нолики используют Dueling DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODgwMy42NDM0MzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3OTc4NzM2Ljg0Mzk2NjIiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk3ODQ2NS4yMDI3MzczIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODc1NC41NjgxMzM0Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzk3ODM4Ni45MDU1NzciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5Nzg1NzkuMjE5MjgyOSI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5Nzg2NDYuODI4OTQ2OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5Nzg1MDcuMTY0NjA2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTc4NDY4Ljk0NTU4NzYiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODgwMy42NDM0MzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3OTc4NzM2Ljg0Mzk2NjIiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk3ODQ2NS4yMDI3MzczIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODc1NC41NjgxMzM0Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzk3ODM4Ni45MDU1NzciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5Nzg1NzkuMjE5MjgyOSI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5Nzg2NDYuODI4OTQ2OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5Nzg1MDcuMTY0NjA2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTc4NDY4Ljk0NTU4NzYiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999&regexInput=%5E0_)


Тут оказалось сложным обучить модель с Dueling DQN за ноликов с точки зрения побед. По большой части наблюдается рост числа ничьих. Победы растут, но не быстро. Тут либо надо больше времени на обучение, либо другие гиперпараметры выбрать.

### Окружение 4x4

#### Крестики используют Dueling DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/ZeSdQLG2SxyAVnvwzD1mIg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/ZeSdQLG2SxyAVnvwzD1mIg/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODA0ODg0MC41NTM3MDgiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2MzgwNDg3ODUuODM3MTIzOSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MDQ4ODQxLjI4NDY5MDEiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM4MDQ4ODAxLjc4MTc4NTUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM4MDQ4NjQyLjY4OTkyNDciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2MzgwNDg5MjEuNjYyMDI3NiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2MzgwNDg5MDguMjg3MTI1Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzODA0ODQxMi4zMDcwMTc2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM4MDQ4Nzc3LjA5NTg3MjIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/ZeSdQLG2SxyAVnvwzD1mIg/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODA0ODg0MC41NTM3MDgiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2MzgwNDg3ODUuODM3MTIzOSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MDQ4ODQxLjI4NDY5MDEiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM4MDQ4ODAxLjc4MTc4NTUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM4MDQ4NjQyLjY4OTkyNDciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2MzgwNDg5MjEuNjYyMDI3NiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2MzgwNDg5MDguMjg3MTI1Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzODA0ODQxMi4zMDcwMTc2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM4MDQ4Nzc3LjA5NTg3MjIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999&regexInput=%5E1_)

#### Нолики используют Dueling DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/zworinzQRiOp4latZHklqg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/zworinzQRiOp4latZHklqg/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODEwNjM5Ny41Nzg4NDgiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2MzgxMDY0MjkuNTExMzkzNSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MTA2Mzk4Ljc4Mjg0MjIiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM4MTA2MzQ2LjAxNTkxOTIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM4MTA2NDI4LjIyNjM2OCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzODEwNjQxOS43MDk5NTU3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzODEwNjI1OC4wNDExNjIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM4MTA2MjI4Ljk5NDQ5ODciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzLzE2MzgxMDYyNTIuMzA0MDUxNiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/zworinzQRiOp4latZHklqg/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODEwNjM5Ny41Nzg4NDgiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2MzgxMDY0MjkuNTExMzkzNSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MTA2Mzk4Ljc4Mjg0MjIiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM4MTA2MzQ2LjAxNTkxOTIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM4MTA2NDI4LjIyNjM2OCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzODEwNjQxOS43MDk5NTU3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzODEwNjI1OC4wNDExNjIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM4MTA2MjI4Ljk5NDQ5ODciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzLzE2MzgxMDYyNTIuMzA0MDUxNiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999&regexInput=%5E1_)

# Часть третья: расширим и углубим поиск

Далее приведены результаты игры разных стратегий между собой в том числе и с MCTS деревом при различных комбинациях rollout. 

MCTS дерево реализована в режиме online. Можно обновлять статистики по мере игры или заранее сыграть несколько игр, а потом использовать дерево обновляя статистики в реальном времени. В качестве критерия для выбора узлов на этапе selection используется UCB1. 

Для окружения 4x4 оказалось вычислительно сложно посчитать много эпизодов для сбора достаточной статистики. Пришлось использовать небольшое число эпизодов для предварительной оценки состояний и действий.

Стратегии игроков:

* dqn - DQN обучение
* dueling-dqn - Dueling DQN обучение
* random - случайная стратегия
* q-table - табличное Q-обучение
* mcts-rollout-* - это MCTS дерево, которое при rollout использует соответствующую стратегию.
* rand_minmax - это рандомизированное минимакс дерево как указано в самом начале. Только для окружения 3x3 представлено.

## Для окружения 3x3

In [86]:
stat_dir = pathlib.Path("./stat/game_simul/3")

In [87]:
game_stat_3x3 = game_stat_to_table(stat_dir)

In [88]:
cross_win_col_name = "Доля побед крестиков"
top_n = 10

Максимальная доля побед за крестиков

In [89]:
game_stat_3x3.nlargest(n=top_n, columns=cross_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
48,q-table_vs_dqn,0.9880,0.0010,0.0110
27,mcts-rollout-dueling-dqn_vs_dueling-dqn,0.9870,0.0110,0.0020
12,q-table_vs_mcts-rollout-random,0.9840,0.0035,0.0125
21,q-table_vs_mcts-rollout-q-table,0.9830,0.0020,0.0150
40,q-table_vs_mcts-rollout-dueling-dqn,0.9825,0.0030,0.0145
25,q-table_vs_dueling-dqn,0.9820,0.0010,0.0170
14,q-table_vs_random,0.9815,0.0035,0.0150
50,dqn_vs_dueling-dqn,0.9760,0.0060,0.0180
35,mcts-rollout-random_vs_dueling-dqn,0.9720,0.0250,0.0030
6,rand-minmax_vs_mcts-rollout-dueling-dqn,0.9325,0.0305,0.0370


In [90]:
circle_win_col_name =  "Доля побед ноликов"

Максимальная доля побед за ноликов

In [91]:
game_stat_3x3.nlargest(n=top_n, columns=circle_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
36,mcts-rollout-q-table_vs_q-table,0.1810,0.6745,0.1445
56,random_vs_rand-minmax,0.1720,0.6560,0.1720
39,random_vs_q-table,0.1880,0.5945,0.2175
8,mcts-rollout-q-table_vs_rand-minmax,0.2685,0.4790,0.2525
55,random_vs_dqn,0.1370,0.4240,0.4390
22,mcts-rollout-random_vs_rand-minmax,0.3230,0.3970,0.2800
24,mcts-rollout-dueling-dqn_vs_rand-minmax,0.3370,0.3900,0.2730
60,mcts-rollout-q-table_vs_dqn,0.1985,0.3470,0.4545
9,mcts-rollout-random_vs_q-table,0.5485,0.3345,0.1170
13,random_vs_mcts-rollout-random,0.5690,0.2995,0.1315


In [92]:
draw_col_name = "Доля ничьих"

Максимальная доля ничьих

In [93]:
game_stat_3x3.nlargest(n=top_n, columns=draw_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
16,dqn_vs_dqn,0.0060,0.0080,0.9860
28,dueling-dqn_vs_q-table,0.0115,0.0085,0.9800
42,dueling-dqn_vs_dqn,0.0210,0.0000,0.9790
52,q-table_vs_q-table,0.0250,0.0070,0.9680
18,dqn_vs_q-table,0.0175,0.0150,0.9675
46,dueling-dqn_vs_dueling-dqn,0.0365,0.0010,0.9625
26,dueling-dqn_vs_rand-minmax,0.3230,0.0130,0.6640
51,rand-minmax_vs_q-table,0.0625,0.2810,0.6565
61,mcts-rollout-dueling-dqn_vs_dqn,0.2420,0.1020,0.6560
38,rand-minmax_vs_dueling-dqn,0.2600,0.1110,0.6290


In [94]:
game_stat_3x3[game_stat_3x3["Стратегии игроков"].str.contains("mcts")]

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
15,dqn_vs_mcts-rollout-dueling-dqn,0.7075,0.0245,0.2680
37,dqn_vs_mcts-rollout-q-table,0.7020,0.0170,0.2810
43,dqn_vs_mcts-rollout-random,0.7060,0.0230,0.2710
54,dueling-dqn_vs_mcts-rollout-dueling-dqn,0.7575,0.0125,0.2300
53,dueling-dqn_vs_mcts-rollout-q-table,0.7670,0.0075,0.2255
3,dueling-dqn_vs_mcts-rollout-random,0.7665,0.0120,0.2215
61,mcts-rollout-dueling-dqn_vs_dqn,0.2420,0.1020,0.6560
27,mcts-rollout-dueling-dqn_vs_dueling-dqn,0.9870,0.0110,0.0020
47,mcts-rollout-dueling-dqn_vs_mcts-rollout-dueli...,0.9055,0.0330,0.0615
41,mcts-rollout-dueling-dqn_vs_mcts-rollout-q-table,0.8830,0.0480,0.0690


In [95]:
HTML(game_stat_3x3.to_html())

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
16,dqn_vs_dqn,0.0060,0.0080,0.9860
50,dqn_vs_dueling-dqn,0.9760,0.0060,0.0180
15,dqn_vs_mcts-rollout-dueling-dqn,0.7075,0.0245,0.2680
37,dqn_vs_mcts-rollout-q-table,0.7020,0.0170,0.2810
43,dqn_vs_mcts-rollout-random,0.7060,0.0230,0.2710
18,dqn_vs_q-table,0.0175,0.0150,0.9675
17,dqn_vs_rand-minmax,0.3260,0.0675,0.6065
63,dqn_vs_random,0.6915,0.0240,0.2845
42,dueling-dqn_vs_dqn,0.0210,0.0000,0.9790
46,dueling-dqn_vs_dueling-dqn,0.0365,0.0010,0.9625


## Для окружения 4x4

In [96]:
stat_dir = pathlib.Path("./stat/game_simul/4")

In [97]:
del game_stat_3x3

In [98]:
game_stat_4x4 = game_stat_to_table(stat_dir)

In [99]:
cross_win_col_name = "Доля побед крестиков"
top_n = 10

Максимальная доля побед за крестиков

In [100]:
game_stat_4x4.nlargest(n=top_n, columns=cross_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
25,q-table_vs_q-table,0.4855,0.2640,0.2505
19,dqn_vs_mcts-rollout-q-table,0.4465,0.0400,0.5135
21,dqn_vs_mcts-rollout-dqn,0.4435,0.0350,0.5215
35,dqn_vs_random,0.4380,0.0420,0.5200
22,dqn_vs_mcts-rollout-random,0.4320,0.0350,0.5330
11,q-table_vs_mcts-rollout-q-table,0.3945,0.1665,0.4390
8,dqn_vs_q-table,0.3920,0.0030,0.6050
10,q-table_vs_mcts-rollout-dqn,0.3855,0.1640,0.4505
3,q-table_vs_mcts-rollout-random,0.3765,0.1550,0.4685
26,mcts-rollout-dqn_vs_mcts-rollout-q-table,0.3660,0.2350,0.3990


Максимальная доля побед за ноликов

In [101]:
circle_win_col_name =  "Доля побед ноликов"
game_stat_4x4.nlargest(n=top_n, columns=circle_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
2,mcts-rollout-random_vs_q-table,0.3295,0.4185,0.2520
28,random_vs_dqn,0.0455,0.3660,0.5885
32,mcts-rollout-q-table_vs_dqn,0.0315,0.3430,0.6255
13,mcts-rollout-dqn_vs_q-table,0.3605,0.3275,0.3120
12,mcts-rollout-dqn_vs_dqn,0.0345,0.2975,0.6680
20,random_vs_q-table,0.3150,0.2945,0.3905
27,mcts-rollout-random_vs_mcts-rollout-dqn,0.2920,0.2925,0.4155
24,mcts-rollout-random_vs_random,0.3040,0.2865,0.4095
9,mcts-rollout-random_vs_mcts-rollout-q-table,0.2910,0.2805,0.4285
31,mcts-rollout-random_vs_mcts-rollout-random,0.3005,0.2785,0.4210


In [102]:
draw_col_name = "Доля ничьих"

Максимальная доля ничьих

In [103]:
game_stat_4x4.nlargest(n=top_n, columns=draw_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
7,dqn_vs_dqn,0.0020,0.0030,0.9950
23,q-table_vs_dqn,0.0105,0.1355,0.8540
14,mcts-rollout-random_vs_dqn,0.0390,0.2640,0.6970
12,mcts-rollout-dqn_vs_dqn,0.0345,0.2975,0.6680
32,mcts-rollout-q-table_vs_dqn,0.0315,0.3430,0.6255
8,dqn_vs_q-table,0.3920,0.0030,0.6050
28,random_vs_dqn,0.0455,0.3660,0.5885
22,dqn_vs_mcts-rollout-random,0.4320,0.0350,0.5330
21,dqn_vs_mcts-rollout-dqn,0.4435,0.0350,0.5215
35,dqn_vs_random,0.4380,0.0420,0.5200


In [104]:
HTML(game_stat_4x4.to_html())

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
7,dqn_vs_dqn,0.0020,0.0030,0.9950
21,dqn_vs_mcts-rollout-dqn,0.4435,0.0350,0.5215
19,dqn_vs_mcts-rollout-q-table,0.4465,0.0400,0.5135
22,dqn_vs_mcts-rollout-random,0.4320,0.0350,0.5330
8,dqn_vs_q-table,0.3920,0.0030,0.6050
35,dqn_vs_random,0.4380,0.0420,0.5200
12,mcts-rollout-dqn_vs_dqn,0.0345,0.2975,0.6680
34,mcts-rollout-dqn_vs_mcts-rollout-dqn,0.3255,0.2440,0.4305
26,mcts-rollout-dqn_vs_mcts-rollout-q-table,0.3660,0.2350,0.3990
29,mcts-rollout-dqn_vs_mcts-rollout-random,0.3495,0.2435,0.4070
